# xDSL-MLIR interoperation tutorial

This tutorial aims to showcase a simple pipeline of actions to unlock MLIR optimisations when lowering from xDSL.
This tutorial can help users getting familiar with the xDSL-MLIR interoperation. We will start from a higher level of xDSL abstraction, lower to MLIR generic format, apply an optimisation and the return to xDSL-land.

# Problem setup

We start by writing a simple example consisting of adding integers. 
We are writing this example using constructs that are supported in xDSL.

We create 4 integers, namely a, b, c, d.
Then we just accumulate by the simple following pseudocode and print the result:


```bash
a = 1
b = 2
c = a + b
d = a + b
e = c + d
print(e)
```


In [1]:
from xdsl.dialects.arith import AddiOp, ConstantOp
from xdsl.dialects.builtin import IntegerAttr, ModuleOp, i32
from xdsl.dialects.vector import PrintOp
from xdsl.ir import Block, Region

# Define two integer constants
a = ConstantOp(IntegerAttr.from_int_and_width(1, 32), i32)
b = ConstantOp(IntegerAttr.from_int_and_width(2, 32), i32)

# Operations on these constants
c = AddiOp(a, b)
d = AddiOp(a, b)
e = AddiOp(c, d)
f = PrintOp.get(e)

# Create Block from operations and Region from blocks
block0 = Block([a, b, c, d, e, f])
region0 = Region(block0)

# Create an Operation from the region
op = ModuleOp(region0)

Using xDSLs printer we can print this operation.
For convenience we provide a file called `source.mlir` with the code printed below

In [2]:
from xdsl.printer import Printer

# Print in xdsl format
printer = Printer()
printer.print(op)

builtin.module {
  %0 = arith.constant 1 : i32
  %1 = arith.constant 2 : i32
  %2 = arith.addi %0, %1 : i32
  %3 = arith.addi %0, %1 : i32
  %4 = arith.addi %2, %3 : i32
  "vector.print"(%4) : (i32) -> ()
}


In [3]:
# Cross-check file content
!cat source.mlir

"builtin.module"() ({
  %0 = "arith.constant"() {value = 1 : i32} : () -> i32
  %1 = "arith.constant"() {value = 2 : i32} : () -> i32
  %2 = "arith.addi"(%0, %1) : (i32, i32) -> i32
  %3 = "arith.addi"(%0, %1) : (i32, i32) -> i32
  %4 = "arith.addi"(%2, %3) : (i32, i32) -> i32
}) : () -> ()


Now lets try to benefit from some mlir optimisation.
For this example we will use the [Common subexpression elimination](https://en.wikipedia.org/wiki/Common_subexpression_elimination).

See some documentation here: [mlir.llvm CSE docs](https://mlir.llvm.org/docs/Passes/#-cse-eliminate-common-sub-expressions)

Assuming you have already mlir-opt installed in your machine you can apply the CSE optimisation using the folloing command:

In [4]:
!mlir-opt source.mlir -cse --mlir-print-op-generic -o opt-out.mlir
!cat opt-out.mlir

"builtin.module"() ({
  %0 = "arith.constant"() <{value = 1 : i32}> : () -> i32
  %1 = "arith.constant"() <{value = 2 : i32}> : () -> i32
  %2 = "arith.addi"(%0, %1) <{overflowFlags = #arith.overflow<none>}> : (i32, i32) -> i32
}) : () -> ()



We can clearly see in the optimised output that after CSE we do not need to calculate:

```
"arith.addi"(%0, %1) : (i32, i32) -> i32
```

twice! Now can we back to xDSL? Yes we can!

In [5]:
!./../xdsl/tools/xdsl-opt opt-out.mlir -f mlir -t mlir -o ret.mlir
!cat ret.mlir

builtin.module {
  %0 = arith.constant 1 : i32
  %1 = arith.constant 2 : i32
  %2 = arith.addi %0, %1 : i32
}

